<a href="https://colab.research.google.com/github/lobasfs/-ML-/blob/main/Regression_CC50_LobasFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [12]:
data = pd.read_csv('Обработанный.csv')
data.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,2,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Разделяем данные на признаки и целевой столбец
X = data.drop(['CC50, mM', 'SI','IC50, mM'], axis=1)
y = np.log(data['CC50, mM'])
X

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,340.300,384.350449,158,...,0,0,0,0,0,0,0,0,3,0
1,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,340.300,388.381750,162,...,0,0,0,0,0,0,0,0,3,0
2,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,388.344,446.458903,186,...,2,0,0,0,0,0,0,0,3,0
3,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,352.311,398.366099,164,...,0,0,0,0,0,0,0,0,4,0
4,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,424.377,466.334799,184,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,12.934891,12.934891,0.048029,-0.476142,0.382752,49.133333,414.542,380.270,414.240624,164,...,0,0,0,0,0,0,0,0,0,0
945,13.635345,13.635345,0.030329,-0.699355,0.369425,44.542857,485.621,446.309,485.277738,192,...,0,0,0,0,0,0,0,0,0,0
946,13.991690,13.991690,0.026535,-0.650790,0.284923,41.973684,545.742,502.398,545.281109,210,...,0,1,0,0,0,0,0,0,0,0
947,13.830180,13.830180,0.146522,-1.408652,0.381559,39.000000,522.635,480.299,522.282883,208,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Проводим стандартизацию признаков
scales = StandardScaler()
X_scaled = scales.fit_transform(X)
X_scaled

array([[-1.7555751 , -1.7555751 ,  1.21623547, ..., -0.27782192,
         2.2416977 , -0.08620324],
       [-2.09860733, -2.09860733,  2.08224504, ..., -0.27782192,
         2.2416977 , -0.08620324],
       [-2.50270017, -2.50270017,  2.13753458, ..., -0.27782192,
         2.2416977 , -0.08620324],
       ...,
       [ 0.939063  ,  0.939063  , -0.91383016, ..., -0.27782192,
        -0.1711996 , -0.08620324],
       [ 0.89014984,  0.89014984, -0.2052462 , ..., -0.27782192,
        -0.1711996 , -0.08620324],
       [ 0.75407414,  0.75407414, -1.05621383, ..., -0.27782192,
        -0.1711996 , -0.08620324]])

In [15]:
# Отбор 30 лучших признаков по mutual_info, для понижения размерности ускорения обучения
selector = SelectKBest(score_func=mutual_info_regression, k=30)
X_selected = selector.fit_transform(X_scaled, y)
selected_feature_names = X.columns[selector.get_support()]
selected_feature_names

Index(['SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons',
       'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge',
       'MinAbsPartialCharge', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
       'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'BertzCT',
       'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'HallKierAlpha',
       'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'HeavyAtomCount', 'MolMR'],
      dtype='object')

In [16]:
X_selected

array([[1.07127087, 0.24826084, 0.09240052, ..., 0.47161129, 0.32963129,
        0.75004942],
       [1.25144331, 0.27971161, 0.09240052, ..., 0.49626096, 0.32963129,
        0.72068484],
       [1.01285559, 0.73309568, 0.48379975, ..., 0.99147747, 0.7660737 ,
        1.23599791],
       ...,
       [0.99600147, 1.50480961, 1.41296151, ..., 1.53586704, 1.42073732,
        1.41724796],
       [0.7615995 , 1.3245683 , 1.23292797, ..., 1.32284623, 1.31162671,
        1.04231688],
       [1.51679807, 0.57544401, 0.48338427, ..., 0.70654988, 0.6569631 ,
        0.64822388]])

In [17]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [18]:
# Возьмем пять моделей, обычную линейную регрессию, регуляризацию Ridge и Lasso,
# RandomForest, XGBoost и их разные гиперпараметры для подбора
models = {
    "LinearRegression": (LinearRegression(), {}),
    "Ridge": (Ridge(), {"alpha": [0.01, 0.1, 1.0, 10.0, 100.0]}),
    "Lasso": (Lasso(max_iter=10000), {"alpha": [0.0001, 0.001, 0.01, 0.1, 1.0]}),
    "RandomForest": (
        RandomForestRegressor(random_state=42),
        {
            "n_estimators": [50, 100, 200],
            "max_depth": [5, 10, 15],
            "min_samples_split": [2, 5]
        }
    ),
    "XGBoost": (
        XGBRegressor(random_state=42, verbosity=0),
        {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 6, 10],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 1.0]
        }
    )
}

In [19]:
# Обучение и сравнение моделей
results = []
for name, (model, params) in models.items():
    grid = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    results.append({
        "Model": name,
        "Best Params": grid.best_params_,
        "MAE:": mean_absolute_error(y_test, y_pred),
        "MSE:": mean_squared_error(y_test, y_pred),
        "R2:": r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
results_df

,Model,Best Params,MAE:,MSE:,R2:
0,LinearRegression,{},1.267024,2.837647,0.091474
1,Ridge,{'alpha': 100.0},1.323488,2.959274,0.052533
2,Lasso,{'alpha': 0.1},1.371685,3.092294,0.009944
3,RandomForest,"{'max_depth': 10, 'min_samples_split': 5, 'n_e...",0.965400,1.835271,0.412404
4,XGBoost,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",1.009425,1.877053,0.399026


In [10]:
rf = RandomForestRegressor(max_depth=10, min_samples_split = 5, n_estimators=200, random_state=42)

rf.fit(X_train, y_train)

y_rf_pred = rf.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_rf_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_rf_pred))
print('R2 score:', r2_score(y_test, y_rf_pred))

Mean Absolute Error: 0.8606519616464356
Mean Squared Error: 1.8753227751843558
R2 score: 0.39958015694144655


In [11]:
xgb =  XGBRegressor(learning_rate =  0.1, max_depth = 3, n_estimators=50, subsample=0.8, random_state=42, verbosity=0)

xgb.fit(X_train, y_train)

y_xgb_pred = xgb.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_xgb_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_xgb_pred))
print('R2 score:', r2_score(y_test, y_xgb_pred))

Mean Absolute Error: 0.8850911496343686
Mean Squared Error: 1.7103920339684517
R2 score: 0.45238583448495584
